In [ ]:
# -*- coding: utf-8 -*-
"""
DEMANDA ELÉCTRICA HORARIA – MODELO HÍBRIDO (Ridge + Boosting) CON DATOS SIMULADOS
=================================================================================

OBJETIVO
--------
Pronosticar demanda eléctrica **horaria** por **subárea** para los próximos **90 días (~3 meses)**,
usando 5 años de historia simulada. Se implementa un **modelo híbrido**:

(A) **Regresión Ridge con Fourier + ARX (Dynamic Harmonic Regression)**
    - Captura estacionalidades **diaria** (24 h) y **semanal** (168 h) mediante términos de
      **Fourier**: senos/cosenos con distintos armónicos. Esto aproxima ciclos periódicos:
        y_t ≈ β0 + Σ_k [a_k sin(2πk t / P) + b_k cos(2πk t / P)] + ...
      donde P es el período (24 o 168) y k=1..K son los armónicos elegidos.
    - Incluye **lags autoregresivos** (ARX) como y_{t-1}, y_{t-24}, y_{t-168} para capturar inercia
      de corto/medio plazo, y **rollings** (promedios móviles) como componente de tendencia suave.
    - Añade **exógenas**: temperatura y festivos.
    - Se usa **Ridge** (L2) para estabilizar al tener muchas features (evita sobreajuste).

(B) **Corrección de residuos con ML (Boosting)**
    - Los residuos del modelo (A) pueden contener **no linealidades**/interacciones no capturadas.
    - Un **HistGradientBoostingRegressor** aprende a predecir dichos residuos usando exógenas
      y calendario (si se añade), y la predicción final es:
        ŷ_t(final) = ŷ_t(A) + ŷ_t_residuos(ML)

VALIDACIÓN WALK-FORWARD
-----------------------
- Se evalúa en los **últimos 90 días** con un **pronóstico recursivo**: cada hora predicha alimenta
  los lags de las horas futuras (como en operación real). Métricas: **MAPE** y **RMSE**.

TEMPERATURA FUTURA EN PRODUCCIÓN
--------------------------------
- Aquí se simula temperatura futura a partir de promedios (doy, hour) del último año.
- En tu caso real: sustituir por un **pronóstico meteorológico** (Meteostat/NOAA/OpenWeatherMap).

CÓMO ADAPTAR A TUS DATOS
------------------------
1) Reemplaza la función `build_panel()` por tu dataframe real (timestamp, subarea, demand, temp, holiday).
2) Mantén `fourier_terms`, lags, rollings.
3) Entrena y evalúa con `fit_models_for_subarea`, `walkforward_test`, y luego `recursive_forecast`.
4) Ajusta hiperparámetros (K_DAILY/K_WEEKLY, lags, alpha de Ridge, profundidad del Boosting).
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import pi

from sklearn.linear_model import RidgeCV
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error


# =========================
# 0) PARÁMETROS GENERALES
# =========================
RANDOM_SEED   = 42
np.random.seed(RANDOM_SEED)

N_SUBAREAS    = 6
START_DATE    = "2020-01-01 00:00:00"
END_DATE      = "2024-12-31 23:00:00"   # ~5 años de historia
TEST_DAYS     = 90                      # ventana de evaluación (walk-forward)
FORECAST_DAYS = 90                      # horizonte futuro ≈ 3 meses

# Estacionalidades
DAILY_PERIOD  = 24                      # ciclo diario
WEEKLY_PERIOD = 24 * 7                  # ciclo semanal
K_DAILY       = 3                       # nº de armónicos diarios
K_WEEKLY      = 2                       # nº de armónicos semanales

# Lags ARX
LAGS = [1, 24, 168]                     # 1h, 1 día, 1 semana

# =========================
# 1) SIMULACIÓN DE DATOS
# =========================
def simulate_temperature(idx, subarea_id):
    """
    Base teórica:
    -------------
    La temperatura se modela con un componente **anual** (seno), un **diario** (seno suave),
    un pequeño sesgo por subárea y ruido blanco. Esto produce estacionalidad climática realista.
    """
    dayofyear = idx.dayofyear.values
    hour      = idx.hour.values

    annual = 10 * np.sin(2 * pi * dayofyear / 365.25 - 0.8)      # amplitud ±10°C
    daily  =  2 * np.sin(2 * pi * hour / 24.0 - 0.5)             # ciclo diario suave
    bias   = (subarea_id - 3) * 0.3                              # sesgo por subárea
    noise  = np.random.normal(0, 1.5, size=len(idx))

    return 18 + annual + daily + bias + noise                     # base 18°C

def simulate_demand(idx, subarea_id, temp):
    """
    Base teórica:
    -------------
    La demanda agrega:
    - Base por subárea
    - Estacionalidad diaria (picos en mañana/noche) y semanal (fin de semana reduce)
    - Estacionalidad anual suave
    - Efecto temperatura (HDD/CDD respecto a 18°C)
    - Festivos (reducción)
    - Ruido blanco

    Con esto, la relación demanda-temperatura no es lineal pura (heating/cooling degree).
    """
    hour = idx.hour.values
    dow  = idx.dayofweek.values
    doy  = idx.dayofyear.values

    base   = 220 + 20 * (subarea_id - 3) + np.random.normal(0, 5)  # nivel por subárea
    daily  = 20 * np.sin(2 * pi * (hour - 8)  / 24.0) + 15 * np.sin(2 * pi * (hour - 19) / 24.0)
    weekly = -25 * (dow >= 5).astype(float)                         # fin de semana menor
    annual = 10 * np.cos(2 * pi * doy / 365.25)

    # Festivos fijos por año (ejemplo)
    holidays = pd.Series(0, index=idx)
    for y in np.unique(idx.year):
        for m, d in [(1,1), (5,1), (12,25)]:
            try:
                holidays[pd.Timestamp(y, m, d)] = 1
            except:
                pass
    holidays = holidays.reindex(idx, fill_value=0).values
    holiday_effect = -30 * holidays

    # Efecto temperatura no lineal (HDD/CDD) alrededor de 18°C
    hdd = np.maximum(0, 18 - temp)     # heating degree days
    cdd = np.maximum(0, temp - 18)     # cooling degree days
    temp_effect = 1.2 * hdd + 1.6 * cdd

    noise = np.random.normal(0, 5, size=len(idx))
    demand = base + daily + weekly + annual + temp_effect + holiday_effect + noise
    return np.maximum(demand, 30), holidays   # piso mínimo

def build_panel():
    """
    Genera un panel (long) con:
      timestamp | subarea | temp | demand | holiday
    para las 6 subáreas y 5 años horarios.
    """
    idx = pd.date_range(START_DATE, END_DATE, freq="H")
    dfs = []
    for s in range(1, N_SUBAREAS + 1):
        temp = simulate_temperature(idx, s)
        demand, holidays = simulate_demand(idx, s, temp)
        df = pd.DataFrame({
            "timestamp": idx,
            "subarea": f"S{s}",
            "temp": temp,
            "demand": demand,
            "holiday": holidays
        })
        dfs.append(df)
    return pd.concat(dfs, ignore_index=True)

# Construcción del dataset simulado
data = build_panel().sort_values(["subarea", "timestamp"]).reset_index(drop=True)


# ============================================
# 2) FEATURE ENGINEERING (FOURIER + LAGS + ROLL)
# ============================================
def fourier_terms(timestamps, period, K):
    """
    Base teórica:
    -------------
    Los términos de Fourier (sin/cos) aproximan **estacionalidades periódicas** sin crear
    24 ó 168 dummies. Con K armónicos, capturamos distintas frecuencias dentro del período.
    """
    t = np.arange(len(timestamps))
    out = {}
    for k in range(1, K + 1):
        out[f"sin_{period}_{k}"] = np.sin(2 * pi * k * t / period)
        out[f"cos_{period}_{k}"] = np.cos(2 * pi * k * t / period)
    return pd.DataFrame(out, index=timestamps)

def add_lags(group_df, target_col="demand", lags=LAGS):
    """
    Base teórica:
    -------------
    Los lags ARX incorporan **memoria** del sistema: la demanda depende de su propio pasado
    (y_{t-1}, y_{t-24}, y_{t-168}). Se añaden también **promedios móviles** como tendencia suave.
    """
    for L in lags:
        group_df[f"lag_{L}"] = group_df[target_col].shift(L)
    group_df["roll24"]  = group_df[target_col].rolling(24,  min_periods=1).mean()
    group_df["roll168"] = group_df[target_col].rolling(168, min_periods=1).mean()
    return group_df

# Aplicar lags por subárea (evita mezclar información entre grupos)
data = data.groupby("subarea", group_keys=False).apply(add_lags)

# Fourier diarios y semanales sobre la malla temporal completa
unique_ts = data["timestamp"].drop_duplicates()
ft_daily  = fourier_terms(unique_ts, DAILY_PERIOD,  K_DAILY)
ft_weekly = fourier_terms(unique_ts, WEEKLY_PERIOD, K_WEEKLY)
ft_all    = pd.concat([ft_daily, ft_weekly], axis=1).reset_index().rename(columns={"index": "timestamp"})

# Incorporar Fourier al panel
data = data.merge(ft_all, on="timestamp", how="left")


# =========================
# 3) SPLIT TEMPORAL
# =========================
last_ts   = data["timestamp"].max()
test_start = last_ts - pd.Timedelta(days=TEST_DAYS) + pd.Timedelta(hours=1)

# =========================
# 4) MATRICES X / y
# =========================
ridge_features = (
    ["temp", "holiday", "roll24", "roll168"] +
    [c for c in data.columns if c.startswith("sin_") or c.startswith("cos_")] +
    [f"lag_{L}" for L in LAGS]
)

def prepare_xy(df):
    """
    Construye X (features) e y (objetivo) y descarta filas con NaN (por lags iniciales).
    """
    X = df[ridge_features].copy()
    y = df["demand"].copy()
    valid = ~X.isna().any(axis=1) & y.notna()
    return X[valid], y[valid], valid

# ==========================================
# 5) ENTRENAMIENTO Y PRONÓSTICO POR SUBÁREA
# ==========================================
def fit_models_for_subarea(df_sub):
    """
    Ajusta:
      (A) RidgeCV sobre Fourier + ARX + exógenas
      (B) Boosting (HGBR) sobre RESIDUOS del (A), con features exógenas (temp, holiday).
    """
    train_mask = df_sub["timestamp"] < test_start
    test_mask  = df_sub["timestamp"] >= test_start

    X_train, y_train, vtrain = prepare_xy(df_sub[train_mask])

    # --- Capa A: Ridge con búsqueda de alpha por CV logaritmica
    ridge = RidgeCV(alphas=np.logspace(-3, 3, 13), fit_intercept=True)
    ridge.fit(X_train, y_train)

    # Residuos en train (para entrenar ML de residuos)
    resid_train = y_train - ridge.predict(X_train)

    # --- Capa B: ML para residuos (rápido y robusto)
    ml_features = ["temp", "holiday"]    # en producción puedes añadir dummies de hora/día si lo deseas
    X_train_ml  = df_sub[train_mask].loc[vtrain, ml_features]
    ml = HistGradientBoostingRegressor(random_state=RANDOM_SEED, max_depth=6)
    ml.fit(X_train_ml, resid_train)

    return ridge, ml, {"ml_features": ml_features}

def recursive_forecast(df_sub, ridge, ml, horizon_hours, start_time):
    """
    Pronóstico recursivo a partir de 'start_time' por 'horizon_hours' horas.

    Base teórica:
    -------------
    - En operación, no tenemos el futuro real de y; por eso **retroalimentamos** el pronóstico
      (yhat) a los lags para la siguiente hora. Esta recursividad evita usar "futuros reales".
    - Las features determinísticas (Fourier) y exógenas conocidas (p. ej. festivos programados,
      pronóstico de temperatura) sí están disponibles.
    """
    # Historial hasta start_time (incluye valores reales para lags iniciales)
    hist = df_sub[df_sub["timestamp"] <= start_time].copy().reset_index(drop=True)
    # Índice futuro
    future_index = pd.date_range(start_time + pd.Timedelta(hours=1), periods=horizon_hours, freq="H")
    future = pd.DataFrame({"timestamp": future_index, "subarea": hist["subarea"].iloc[0]})

    # --- Temperatura futura aproximada:
    #     promediamos por (día del año, hora) del último año histórico para esa subárea.
    last_year_mask = hist["timestamp"] >= (hist["timestamp"].max() - pd.Timedelta(days=365))
    last_year = hist.loc[last_year_mask].copy()
    last_year["hour"] = last_year["timestamp"].dt.hour
    last_year["doy"]  = last_year["timestamp"].dt.dayofyear
    grid = last_year.groupby(["doy", "hour"])["temp"].mean().reset_index()

    future["hour"] = future["timestamp"].dt.hour
    future["doy"]  = future["timestamp"].dt.dayofyear
    future = future.merge(grid, on=["doy", "hour"], how="left", suffixes=("", "_mean"))
    future["temp"] = future["temp"].fillna(last_year["temp"].mean())

    # --- Festivos futuros (mismo patrón simple)
    future["holiday"] = 0
    for y in np.unique(future["timestamp"].dt.year):
        for m, d in [(1,1), (5,1), (12,25)]:
            t = pd.Timestamp(y, m, d)
            if t in set(future["timestamp"]):
                future.loc[future["timestamp"] == t, "holiday"] = 1

    # --- Fourier para hist+futuro (mantiene fase coherente)
    full_idx = pd.DatetimeIndex(pd.concat([hist["timestamp"], future["timestamp"]]))
    ft_d = fourier_terms(full_idx, DAILY_PERIOD,  K_DAILY)
    ft_w = fourier_terms(full_idx, WEEKLY_PERIOD, K_WEEKLY)
    ft_full = pd.concat([ft_d, ft_w], axis=1).reset_index().rename(columns={"index": "timestamp"})
    hist   = hist.merge(ft_full, on="timestamp", how="left")
    future = future.merge(ft_full, on="timestamp", how="left")

    # Serie yhat (usamos reales hasta start_time)
    hist["yhat"] = hist["demand"]

    def update_lags_and_rolls(df):
        for L in LAGS:
            df[f"lag_{L}"] = df["yhat"].shift(L)
        df["roll24"]  = df["yhat"].rolling(24,  min_periods=1).mean()
        df["roll168"] = df["yhat"].rolling(168, min_periods=1).mean()
        return df

    hist = update_lags_and_rolls(hist)

    preds      = []
    ridge_cols = [c for c in ridge_features if (c in hist.columns) or (c in future.columns)]
    ml_cols    = ["temp", "holiday"]

    for t in future["timestamp"]:
        row_fut = future[future["timestamp"] == t].copy()
        tmp     = pd.concat([hist, row_fut], ignore_index=True, sort=False)
        tmp     = update_lags_and_rolls(tmp)

        # Capa A (Ridge)
        Xr = tmp.iloc[[-1]][ridge_cols].fillna(method="ffill").fillna(method="bfill")
        yhat_A = ridge.predict(Xr)[0]

        # Capa B (Residuos)
        Xm = tmp.iloc[[-1]][ml_cols].fillna(0)
        yhat_res = ml.predict(Xm)[0]

        yhat = yhat_A + yhat_res

        # Registrar y realimentar para siguientes lags
        new_row = tmp.iloc[[-1]].copy()
        new_row["yhat"]   = yhat
        new_row["demand"] = np.nan
        hist = pd.concat([hist, new_row], ignore_index=True, sort=False)

        preds.append({"timestamp": t, "subarea": hist["subarea"].iloc[0], "yhat": yhat})

    return pd.DataFrame(preds)

def walkforward_test(df_sub, ridge, ml):
    """
    Evalúa en los últimos TEST_DAYS días con pronóstico recursivo (walk-forward).
    Devuelve arrays (y_true, y_pred) para calcular métricas.
    """
    t0 = df_sub.loc[df_sub["timestamp"] >= test_start, "timestamp"].min()
    t1 = df_sub["timestamp"].max()
    horizon_hours = int((t1 - t0) / pd.Timedelta(hours=1)) + 1

    preds = recursive_forecast(df_sub, ridge, ml, horizon_hours, start_time=t0 - pd.Timedelta(hours=1))
    truth = df_sub[(df_sub["timestamp"] >= t0) & (df_sub["timestamp"] <= t1)][["timestamp", "demand"]].reset_index(drop=True)
    out = preds.merge(truth, on="timestamp", how="left")
    return out["demand"].values, out["yhat"].values


# ======================================
# 6) ENTRENAR, EVALUAR Y PRONOSTICAR
# ======================================
metrics_rows   = []
future_forests = []

for sub in data["subarea"].unique():
    df_sub = data[data["subarea"] == sub].copy().reset_index(drop=True)

    # Entrenamiento por subárea
    ridge, ml, meta = fit_models_for_subarea(df_sub)

    # Evaluación walk-forward en TEST
    y_true, y_pred = walkforward_test(df_sub, ridge, ml)
    mape = mean_absolute_percentage_error(y_true, y_pred) * 100
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    metrics_rows.append({"subarea": sub, "MAPE_%": mape, "RMSE": rmse})

    # Pronóstico futuro 90 días
    start_time = df_sub["timestamp"].max()
    horizon_hours = FORECAST_DAYS * 24
    fut = recursive_forecast(df_sub, ridge, ml, horizon_hours, start_time)
    future_forests.append(fut)

# Métricas por subárea y global
metrics_df = pd.DataFrame(metrics_rows)
metrics_df.loc[len(metrics_df)] = {"subarea": "GLOBAL",
                                   "MAPE_%": metrics_df["MAPE_%"].mean(),
                                   "RMSE":   metrics_df["RMSE"].mean()}

# Consolidar pronóstico futuro
forecast_df = pd.concat(future_forests, ignore_index=True).rename(columns={"yhat": "forecast"})

# =========================
# 7) SALIDAS
# =========================
print("\nMÉTRICAS WALK-FORWARD (últimos 90 días):")
print(metrics_df)

print("\nHEAD del pronóstico futuro (primeras 10 filas):")
print(forecast_df.head(10))

# (Opcional) Guardar CSV
# forecast_df.to_csv("forecast_3m_simulado_por_subarea.csv", index=False)

# (Opcional) Gráficas rápidas para una subárea
# sub_demo = "S1"
# df_sub = data[data["subarea"] == sub_demo].copy().reset_index(drop=True)
# ridge, ml, _ = fit_models_for_subarea(df_sub)
# y_true_test, y_pred_test = walkforward_test(df_sub, ridge, ml)
# t0 = df_sub.loc[df_sub["timestamp"] >= test_start, "timestamp"].min()
# t1 = df_sub["timestamp"].max()
# test_index = pd.date_range(t0, t1, freq="H")
# plot_df = pd.DataFrame({"timestamp": test_index, "y_true": y_true_test, "y_pred": y_pred_test})
# last_14 = plot_df["timestamp"] >= (plot_df["timestamp"].max() - pd.Timedelta(days=14))
# plt.figure(figsize=(12,4))
# plt.plot(plot_df[last_14]["timestamp"], plot_df[last_14]["y_true"], label="Real")
# plt.plot(plot_df[last_14]["timestamp"], plot_df[last_14]["y_pred"], label="Pred")
# plt.title(f"Subárea {sub_demo} – Test (últimos 14 días)")
# plt.xlabel("Fecha"); plt.ylabel("Demanda"); plt.legend(); plt.tight_layout(); plt.show()

